In [1]:
import scanpy as sc
data_dir = '../../dataset/ad_ge_imputed_Xenium_breast_cancer_sample1_replicate1.h5ad'
adata = sc.read_h5ad(data_dir)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [2]:
%load_ext autoreload
%autoreload 2
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPBarlowTwins
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/BarlowTwins.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPBarlowTwins(
        gene_dim=19331,  # 根据你的数据调整
        batch_size=128,  # 根据你的需要调整
        units_encoder=units_encoder,
        CHECKPOINT_PATH=ckpt_path
    )


estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [3]:
# 加载 checkpoint
checkpoint = torch.load(ckpt_path)
estim.model.inner_model.load_state_dict({k.replace('backbone.', ''): v for k, v in checkpoint.items() if 'backbone' in k})

# 设定模型为评估模式
estim.model.eval()

MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [4]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [5]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [6]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

samd11      samd11
noc2l        noc2l
klhl17      klhl17
plekhn1    plekhn1
perm1        perm1
            ...   
mt-nd4l    mt-nd4l
mt-nd4      mt-nd4
mt-nd5      mt-nd5
mt-nd6      mt-nd6
mt-cyb      mt-cyb
Name: gene_name, Length: 17693, dtype: object

In [7]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)


In [8]:
existing_genes = adata.var['gene_name']
existing_genes

samd11      samd11
noc2l        noc2l
klhl17      klhl17
plekhn1    plekhn1
perm1        perm1
            ...   
mt-nd4l    mt-nd4l
mt-nd4      mt-nd4
mt-nd5      mt-nd5
mt-nd6      mt-nd6
mt-cyb      mt-cyb
Name: gene_name, Length: 17693, dtype: object

In [9]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 17392
匹配的基因列表: {'cmbl', 'lypd1', 'krtap1-3', 'gbp3', 'hspa4', 'tekt5', 'lrrc58', 'spag8', 'agbl2', 'znf750', 'znf610', 'sgo2', 'lhfpl1', 'alkbh3', 'cbx5', 'nav1', 'ddb1', 'nts', 'gh1', 'uhrf1bp1', 'fndc4', 'c10orf88', 'ptrhd1', 'txlng', 'mef2c', 'zscan21', 'heatr5a', 'epb41l3', 'nrarp', 'cacna1b', 'tbrg1', 'bpifa2', 'b3galt5', 'sv2b', 'ttc22', 'spem3', 'klf10', 'ptpn20', 'znf106', 'fbxo43', 'cfap20', 'argfx', 'lbr', 'bnipl', 'got1', 'nim1k', 'uros', 'zbtb42', 'rspry1', 'c16orf74', 'znf600', 'trmt2b', 'sla', 'dph5', 'ero1b', 'b4gat1', 'henmt1', 'bnc2', 'igdcc3', 'bptf', 'ryr1', 'lrtm2', 'miox', 'il13ra1', 'gcdh', 'calcr', 'tex264', 'nkd2', 'map7', 'mtss2', 'chmp7', 'ddx43', 'slamf9', 'znf444', 'pla2g4a', 'herc2', 'cmas', 'stk32b', 'cox15', 'or5p2', 'tnfrsf10d', 'dgat2', 'snx18', 'unc5d', 'atp8b2', 'klhl17', 'ahsa1', 'lyrm7', 'mpp3', 'angptl5', 'c4orf46', 'dctn3', 'lrrc32', 'cfhr4', 'coprs', 'dhrs7', 'crisp3', 'hhla2', 'c16orf92', 'pfdn2', 'ablim2', 'ints4', 'sult1c3', 'slc3a1',

{'aars',
 'ac004593.3',
 'ac005551.1',
 'ac007906.2',
 'ac008397.1',
 'ac008687.4',
 'ac010255.3',
 'ac010325.1',
 'ac011005.1',
 'ac011195.2',
 'ac013470.2',
 'ac015871.1',
 'ac021072.1',
 'ac021097.2',
 'ac025283.2',
 'ac025287.4',
 'ac067752.1',
 'ac072022.2',
 'ac073111.4',
 'ac087498.1',
 'ac090360.1',
 'ac091057.6',
 'ac092835.1',
 'ac093323.1',
 'ac099489.1',
 'ac104389.5',
 'ac113348.1',
 'ac115220.1',
 'ac118549.1',
 'ac119396.1',
 'ac132217.2',
 'ac134684.8',
 'ac138647.1',
 'ac187653.1',
 'ac233723.1',
 'ac236972.4',
 'acpp',
 'adprhl2',
 'adss',
 'adssl1',
 'al032819.3',
 'al109810.2',
 'al121578.2',
 'al135905.2',
 'al160269.1',
 'al162231.1',
 'al162596.1',
 'al353572.3',
 'al354761.1',
 'al391650.1',
 'al445238.1',
 'al451007.3',
 'al590560.2',
 'al603764.2',
 'al845331.2',
 'alg1l',
 'ap002495.1',
 'arih2os',
 'armc4',
 'arse',
 'atp5md',
 'atp5mpl',
 'atp6ap1l',
 'b3gnt10',
 'bhmg1',
 'bx255925.3',
 'bx276092.9',
 'c10orf142',
 'c10orf55',
 'c11orf40',
 'c11orf74',
 'c

In [10]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [11]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 1939
仅在 all_genes 中存在的基因: {'fsbp', 'eef1a1', 'ntn5', 'rpl27a', 'stip1', 'csh1', 'slc35e2b', 'or10s1', 'nxf2b', 'rrn3', 'pramef4', 'pramef9', 'spink9', 'slc35g6', 'sox11', 'or6v1', 'or10p1', 'defa1', 'scygr5', 'h2ac6', 'hrob', 'bola2b', 'cars1', 'rps3a', 'foxl3', 'uty', 'cfap20dc', 'or51f1', 'or8j3', 'maged4', 'znf506', 'igbp1p2', 'ist1', 'borcs8-mef2b', 'rpl28', 'linc00514', 'pomk', 'ctage15', 'krtap10-9', 'stx16-npepl1', 'htr1b', 'rpl7l1', 'ubr1', 'kifbp', 'tmco5a', 'myo7a', 'timm23b', 'uts2r', 'tas2r41', 'mrpl36', 'ighmbp2', 'krtap10-12', 'or52e2', 'ssu72p4', 'borcs7-asmt', 'mrpl11', 'tardbp', 'daz2', 'usp9y', 'grxcr1', 'or2a2', 'mrps35', 'rpl36a', 'npipb5', 'or10h4', 'cst1', 'chga', 'tas2r60', 'peds1-ube2v1', 'znf418', 'cenpvl1', 'scygr3', 'defb4b', 'or13j1', 'aqp12b', 'dars1', 'rpl34', 'serf1a', 'utp14c', 'or8u1', 'rps27ap5', 'lars1', 'bub1b-pak6', 'rplp2', 'pcdhgb3', 'srgap2b', 'hla-dqb2', 'h4c9', 'nkx2-3', 'dhrs4l2', 'actrt1', 'defa4', 'fam90a18p', 'krtap9-

In [12]:
# 将现有的数据填充到新的矩阵中
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = adata.X[:, i].squeeze()
    else:
        print(f"Gene {gene} not found in all_genes list.")


Gene al391650.1 not found in all_genes list.


Gene yars not found in all_genes list.
Gene adprhl2 not found in all_genes list.


Gene tctex1d4 not found in all_genes list.


Gene tctex1d1 not found in all_genes list.
Gene wdr78 not found in all_genes list.
Gene hhla3 not found in all_genes list.
Gene ac118549.1 not found in all_genes list.


Gene wdr63 not found in all_genes list.


...

Gene nxf5 not found in all_genes list.
Gene glra4 not found in all_genes list.
Gene tmsb15b not found in all_genes list.
Gene h2bfwt not found in all_genes list.
Gene h2bfm not found in all_genes list.
Gene pih1d3 not found in all_genes list.


Gene cxorf56 not found in all_genes list.


Gene fam122b not found in all_genes list.
Gene fam122c not found in all_genes list.
Gene cxorf40a not found in all_genes list.


Gene ac236972.4 not found in all_genes list.


Gene prky not found in all_genes list.


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# 使用 GPU 进行评估（如果可用）
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# 5. 数据集划分（70% 训练，15% 验证，15% 测试）
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42

X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%


X_train_tensor = torch.tensor(X_train).float().to(device)
X_test_tensor = torch.tensor(X_test).float().to(device)
estim.model.to(device)

with torch.no_grad():
    train_embeddings = estim.model.inner_model(X_train_tensor).cpu().numpy()
    test_embeddings = estim.model.inner_model(X_test_tensor).cpu().numpy()

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


cuda


In [14]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np


random_states = [42]

# 循环测试不同的 random_state
for state in random_states:
    print(f"Testing with random_state={state}")
    

    # 初始化和训练KNN分类器
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(train_embeddings, y_train)
    # 模型预测
    
    predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
    accuracy = accuracy_score(y_test, predictions)
    print(f"KNN Accuracy on Test Data: {accuracy}")

    f1 = f1_score(y_test, predictions, average='weighted')
    print(f"Weighted F1 Score: {f1}")
    
    macro_f1 = f1_score(y_test, predictions, average='macro')
    print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
    class_probabilities = np.bincount(y_test) / len(y_test)
    random_accuracy = np.sum(class_probabilities ** 2)
    print(f"Random Guess Accuracy: {random_accuracy}")


    # 生成分类报告
    report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
    print(report)


Testing with random_state=42


KNN Accuracy on Test Data: 0.6822028847300036
Weighted F1 Score: 0.6779740230638085
Macro F1 Score: 0.49867692692585097
Random Guess Accuracy: 0.1320247177116093
                         precision    recall  f1-score   support

                B_Cells       0.68      0.80      0.74       772
           CD4+_T_Cells       0.42      0.50      0.46      1286
           CD8+_T_Cells       0.49      0.48      0.48      1026
                 DCIS_1       0.65      0.66      0.65      1937
                 DCIS_2       0.54      0.53      0.54      1746
            Endothelial       0.86      0.83      0.85      1348
              IRF7+_DCs       0.58      0.41      0.48        74
         Invasive_Tumor       0.78      0.82      0.80      5230
             LAMP3+_DCs       0.52      0.22      0.31        49
          Macrophages_1       0.65      0.73      0.69      1692
          Macrophages_2       0.52      0.35      0.42       223
             Mast_Cells       0.33      0.04      0.06   

In [15]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.inner_model(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_BT_ZS_{random_seed}'] = SSL_embeddings
new_adata.uns[f'BT_ZS_y_test_{random_seed}'] = y_test
new_adata.uns[f'BT_ZS_predictions_{random_seed}'] = predictions
new_adata.uns[f'BT_ZS_target_names_{random_seed}'] = label_encoder.classes_
new_adata.write_h5ad(data_dir)

In [16]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "imputed_Xenium_breast_cancer_sample1_replicate1_barlow_twins_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicor_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['imputed_xenium_breast_cancer_sample1_replicate1'],
    'method': [re.search(r'replicate1_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micor_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.499	0.678	0.682
